In [22]:
from dotenv import  load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

llm = ChatOpenAI(model='gpt-4o')
small_llm = ChatOpenAI(model='gpt-4o-mini')

In [ ]:
from langchain_core.tools import tool

@tool # 해당 함수를 tool 객체로 선언함.
def add(a: int, b:int) -> int:
    # tool에 대한 설명
    """ 
    숫자 a와 b를 더합니다. 
    """
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """ 
    숫자 a와 b를 곱합니다. 
    """
    return a * b

In [21]:
add.invoke({'a':3, 'b':5}) # LLM 없이 tool을 실행함.

8

### 자동 Tool 실행

In [ ]:
from langgraph.prebuilt import ToolNode

tool_list = [add, multiply]

llm_with_tools = small_llm.bind_tools(tool_list) # 해당 LLM에 tool을 바인딩함.
 
tool_node = ToolNode(tool_list) # 해당 tool이 담긴 tool 노드를 생성함.


In [20]:
query = '3 곱하기 5는?'

ai_message = llm_with_tools.invoke(query) # 사용자의 질문을 통해 AI 메시지를 반환함.
ai_message

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_oogBzcEWO8EuE2rC09aMNmdY', 'function': {'arguments': '{"a":3,"b":5}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 84, 'total_tokens': 101, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_644f11dd4d', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-923073d7-f98e-4678-b03c-34ff1dd9a797-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 5}, 'id': 'call_oogBzcEWO8EuE2rC09aMNmdY', 'type': 'tool_call'}], usage_metadata={'input_tokens': 84, 'output_tokens': 17, 'total_tokens': 101, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [18]:
tool_node.invoke({'messages':[ai_message]}) # tool노드와 AI 메시지를 통해 tool메시지를 생성함. (계산 결과만 반환함.)

{'messages': [ToolMessage(content='15', name='multiply', tool_call_id='call_293cD7Zm7OMacByiHdkGhKjG')]}

### 수동 Tool 실행

In [ ]:
tool_list = [add, multiply]

llm_with_tools = small_llm.bind_tools(tool_list) 

In [ ]:
from typing import Sequence
from langchain_core.messages import AnyMessage, HumanMessage

query = '3 곱하기 5는?'

human_message = HumanMessage(query) # 사용자의 질문을 휴먼 메시지로 만듦.

message_list: Sequence[AnyMessage] = [human_message] # 메시지 리스트에 휴먼 메시지를 추가함.


In [ ]:
ai_message = llm_with_tools.invoke(message_list) # 휴먼 메시지를 통해 AI 메시지를 반환함.

In [ ]:
ai_message.tool_calls # tool에 대한 LLM의 판단 결과를 나타냄.

[{'name': 'multiply',
  'args': {'a': 3, 'b': 5},
  'id': 'call_W8PdZ6ogYwo5www8vwLYTDfI',
  'type': 'tool_call'}]

In [ ]:
message_list.append(ai_message) # 메시지 리스트에 AI 메시지를 추가함.

In [ ]:
tool_message = multiply.invoke(ai_message.tool_calls[0]) # tool에 대한 LLM의 판단 결과를 통해 tool메시지를 반환함.

In [ ]:
message_list.append(tool_message) # 메시지 리스트에 tool 메시지를 추가함.

In [ ]:
llm_with_tools.invoke(message_list) # 최종 메시지 리스트를 통해 LLM이 tool을 사용한 결과를 반환함. (계산 결과와 이에 대한 자세한 설명을 반환함.)

AIMessage(content='3 곱하기 5는 15입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 109, 'total_tokens': 121, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_644f11dd4d', 'finish_reason': 'stop', 'logprobs': None}, id='run-b1c293d3-1e29-4968-9423-9ea4ab34962d-0', usage_metadata={'input_tokens': 109, 'output_tokens': 12, 'total_tokens': 121, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})